In [ ]:
# import dependencies
import win32com.client as wc
import xml.etree.ElementTree as ET
import pandas as pd
import mysql.connector as mc
import datetime as dt
from datetime import datetime
import pyodbc
from pdfrw import PdfReader, PdfWriter
import pdfrw 
import os
from pdf2image import convert_from_path
import img2pdf
from PIL import Image

# Get sql user and password
from configTest import mysql_host, mysql_u, mysql_pw
from configTest import vgc_host, vgc_u, vgc_pw
from configTest import svr, db, sql_u, sql_pw

In [ ]:
# Show all columns in DFs
pd.set_option("display.max_columns", None)

In [ ]:
# create batch_id
now = datetime.now()
batch_id = now.strftime("%Y%m%d%H%M%S")

In [ ]:
# PDF Concatenation Function
def ConCat_pdf (file_list, outfn):
    letter_path = './letters/staging/'
    writer = PdfWriter()
    for inputfn in file_list:
        writer.addpages(PdfReader(letter_path + inputfn).pages)

    writer.write(outfn)
    return outfn

In [ ]:
# Delete Function
def delete_file(del_file_path):
    if os.path.exists(del_file_path):
        os.remove(del_file_path)
    else: print (f"{del_file_path} does not exist")

In [ ]:
# Create PDF Function
def fill_pdf(input_pdf_path, output_pdf_path, data_dict):
    ANNOT_KEY = '/Annots'
    ANNOT_FIELD_KEY = '/T'
    ANNOT_VAL_KEY = '/V'
    ANNOT_RECT_KEY = '/Rect'
    SUBTYPE_KEY = '/Subtype'
    WIDGET_SUBTYPE_KEY = '/Widget'

    template_pdf = pdfrw.PdfReader(input_pdf_path)
    
    for page in template_pdf.pages:
        annotations = page[ANNOT_KEY]
        for annotation in annotations:
            if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
                if annotation[ANNOT_FIELD_KEY]:
                    key = annotation[ANNOT_FIELD_KEY][1:-1]
                    if key in data_dict.keys():
                        if type(data_dict[key]) == bool:
                            if data_dict[key] == True:
                                annotation.update(pdfrw.PdfDict(
                                    AS=pdfrw.PdfName('Yes')))
                        else:
                            annotation.update(
                                pdfrw.PdfDict(V='{}'.format(data_dict[key]))
                            )
                            annotation.update(pdfrw.PdfDict(AP=''))
    template_pdf.Root.AcroForm.update(pdfrw.PdfDict(NeedAppearances=pdfrw.PdfObject('true')))

    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)

    # return 1

In [ ]:
# Flatten PDF function
def flatten_pdf(flat_output, img_file):
    # Fillable PDF to Image
    images = convert_from_path(flat_output, dpi=300, size=(2550,3300))
    for i in range(len(images)):
   
    # Save pages as images in the pdf
        images[i].save(img_file + '.png', 'PNG')
    
    # Delete Fillable PDF
    delete_file(img_file + '.pdf')
    
    # opening image
    image_file = Image.open(img_file + '.png')
    
    # Image to Flat PDF
    # define paper size
    letter = (img2pdf.in_to_pt(8.5), img2pdf.in_to_pt(11))
    layout = img2pdf.get_layout_fun(letter)
    # converting into chunks using img2pdf
    pdf_bytes = img2pdf.convert(image_file.filename, layout_fun=layout)
    
    # opening or creating pdf file
    flat_pdf = f"{img_file}.pdf"
    file = open(flat_pdf, "wb")
    
    # writing pdf files with chunks
    file.write(pdf_bytes)
    
    # Add file name to file_name list
    # file_list.append(flat_pdf)

    # closing image file
    image_file.close()
    
     # Delete Fillable PDF
    delete_file(img_file + '.png')

    # closing pdf file
    file.close()

    # return 1

In [ ]:
# GAP Letter function
def gap_letter(template_df, pdf_template, position):
    gap_path = './letters/staging/'

    template_df['payment_amount'] = template_df['payment_amount'].map('${:,.2f}'.format)
    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['StateDesc'] = template_df['StateDesc'].astype(str).replace({'None':''})
    template_df['StateCode'] = template_df['StateCode'].astype(str).replace({'None':''})
    template_df['f_lang'] = template_df['f_lang'].astype(str).replace({'None':''})
    letter_date = f"{datetime.now():%B %d, %Y}"   

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Date': letter_date,
            'Lender': template_df.loc[index]['alt_name'],
            'Contact': template_df.loc[index]['contact'],
            'Address': template_df.loc[index]['address1'],
            'City_St_Zip': f"{template_df.loc[index]['city']}, {template_df.loc[index]['state']} {template_df.loc[index]['zip']}",
            'Lender2': template_df.loc[index]['alt_name'],
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Claim_Nbr': template_df.loc[index]['claim_nbr'],
            'Acct_Nbr': template_df.loc[index]['acct_number'],
            'DOL': template_df.loc[index]['loss_date'],
            'GAP_Amt': template_df.loc[index]['payment_amount'],
            'State': template_df.loc[index]['StateDesc'],
            'St_Code': template_df.loc[index]['StateCode'],
            'Fraud': template_df.loc[index]['f_lang'],
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}-{position}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}-{position}"

        # Flatten pdf using flatten_pdf function
        flatten_pdf(flat_output, img_file) 

    # return 1

In [ ]:
# GAP Calculation function
def calculations(template_df, pdf_template, position):
    gap_path = './letters/staging/'

    template_df['loss_date'] = pd.to_datetime(template_df['loss_date']).dt.strftime('%B %d, %Y')
    template_df['payoff'] = template_df['payoff'].map('${:,.2f}'.format)
    template_df['past_due'] = template_df['past_due'].map('${:,.2f}'.format)
    template_df['late_fees'] = template_df['late_fees'].map('${:,.2f}'.format)
    template_df['skip_pymts'] = template_df['skip_pymts'].map('${:,.2f}'.format)
    template_df['skip_fees'] = template_df['skip_fees'].map('${:,.2f}'.format)
    template_df['primary_pymt'] = template_df['primary_pymt'].map('${:,.2f}'.format)
    template_df['excess_deductible'] = template_df['excess_deductible'].map('${:,.2f}'.format)
    template_df['scr'] = template_df['scr'].map('${:,.2f}'.format)
    template_df['clr'] = template_df['clr'].map('${:,.2f}'.format)
    template_df['cdr'] = template_df['cdr'].map('${:,.2f}'.format)
    template_df['oref'] = template_df['oref'].map('${:,.2f}'.format)
    template_df['salvage'] = template_df['salvage'].map('${:,.2f}'.format)
    template_df['prior_dmg'] = template_df['prior_dmg'].map('${:,.2f}'.format)
    template_df['over_ltv'] = template_df['over_ltv'].map('${:,.2f}'.format)
    template_df['other1_amt'] = template_df['other1_amt'].map('${:,.2f}'.format)
    template_df['other2_amt'] = template_df['other2_amt'].map('${:,.2f}'.format)
    template_df['gap_payable'] = template_df['gap_payable'].map('${:,.2f}'.format)
    template_df['last_payment'] = pd.to_datetime(template_df['last_payment']).dt.strftime('%B %d, %Y')
    template_df['balance_last_pay'] = template_df['balance_last_pay'].map('${:,.2f}'.format)
    template_df['per_day'] = template_df['per_day'].map('${:,.2f}'.format)
    template_df['incp_date'] = pd.to_datetime(template_df['incp_date']).dt.strftime('%B %d, %Y')
    template_df['deductible'] = template_df['deductible'].map('${:,.2f}'.format)
    template_df['subtotal'] = template_df['subtotal'].map('${:,.2f}'.format)
    template_df['nbr_of_days'] = template_df['nbr_of_days'].map('{:,.0f}'.format)
    template_df['interest_rate'] = template_df['interest_rate'].map('{:,.2f}%'.format)
    template_df['ltv'] = template_df['ltv'].map('{:,.2f}%'.format)
    template_df['ltv_limit'] = template_df['ltv_limit'].map('{:,.2f}%'.format)
    template_df['percent_uncovered'] = template_df['percent_uncovered'].map('{:,.2f}%'.format)
    template_df['covered_fin_amount'] = template_df['covered_fin_amount'].map('${:,.2f}'.format)
    template_df['Amt_Fin'] = template_df['Amt_Fin'].map('${:,.2f}'.format)
    template_df['nada_value'] = template_df['nada_value'].map('${:,.2f}'.format)

    for index, row in template_df.iterrows():

        # empty dict
        data_dict = {}
        # store field data in dictionary
        data_dict = {
            'Claim_Number': template_df.loc[index]['claim_nbr'],
            'Status': 'Paid',
            'Borrower': f"{template_df.loc[index]['first']} {template_df.loc[index]['last']}",
            'Vehicle': template_df.loc[index]['vehicle'],
            'Date_Of_Loss': template_df.loc[index]['loss_date'],
            'Type_Of_Loss': template_df.loc[index]['loss_type'],
            'Lender': template_df.loc[index]['alt_name'],         
            'Lender_Contact': template_df.loc[index]['contact'],
            'Insurance_Carrier': template_df.loc[index]['carrier'],
            'Inception_Date': template_df.loc[index]['incp_date'],
            'Deductible': template_df.loc[index]['deductible'],
            'Payoff': template_df.loc[index]['payoff'],
            'Past_Due': template_df.loc[index]['past_due'],
            'Late_Fees': template_df.loc[index]['late_fees'],
            'Skips': template_df.loc[index]['skip_pymts'],
            'Skip_Fees': template_df.loc[index]['skip_fees'],
            'Primary': template_df.loc[index]['primary_pymt'],
            'Deductible_Excess': template_df.loc[index]['excess_deductible'],
            'SCR': template_df.loc[index]['scr'],
            'CL_Refund': template_df.loc[index]['clr'],
            'CD_Refund': template_df.loc[index]['cdr'],
            'O_Refund': template_df.loc[index]['oref'],
            'Salvage': template_df.loc[index]['salvage'],
            'Prior_Damage': template_df.loc[index]['prior_dmg'],
            'Over_LTV': template_df.loc[index]['over_ltv'],
            'Other1_Description': template_df.loc[index]['other1_description'],
            'Other2_Description': template_df.loc[index]['other2_description'],
            'Other1': template_df.loc[index]['other1_amt'],
            'Other2': template_df.loc[index]['other2_amt'],
            'Deduction_Subtotal': template_df.loc[index]['subtotal'],
            'GAP_Amt': template_df.loc[index]['gap_payable'], 
            'Last_pymt_date': template_df.loc[index]['last_payment'], 
            'DOL': template_df.loc[index]['loss_date'],
            'Number_of_days': template_df.loc[index]['nbr_of_days'], 
            'Loan_Payoff_As_of_DOL': template_df.loc[index]['payoff'],
            'Bal_as_of_last_pymt': template_df.loc[index]['balance_last_pay'],
            'Interest_Rate': template_df.loc[index]['interest_rate'],
            'Interest_Per_Day': template_df.loc[index]['per_day'],
            'Amt_financed': template_df.loc[index]['Amt_Fin'],
            'ACV': template_df.loc[index]['nada_value'], 
            'LTV': template_df.loc[index]['ltv'],
            'Max_Amt_Financed': template_df.loc[index]['covered_fin_amount'],
            'LTV_limit': template_df.loc[index]['ltv_limit'],
            'Percentage_Not_Covered': template_df.loc[index]['percent_uncovered']
        }

        # store paths as variables
        output_file = f"{template_df.loc[index]['claim_nbr']}-{position}.pdf"
        output_path_fn = f"{gap_path}{output_file}"

        fill_pdf(pdf_template, output_path_fn, data_dict)

        # Set File Paths
        flat_output = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{output_file}"
        img_file = f"{os.path.dirname(os.path.abspath(output_file))}\{gap_path}\{template_df.loc[index]['claim_nbr']}-{position}"

        # Flatten pdf using flatten_pdf function
        flatten_pdf(flat_output, img_file)

In [ ]:
def clear_dir (path, ext):
    for x in os.listdir(path):
        if x.endswith(ext):
            os.remove(os.path.join(path, x))

In [ ]:
def fileList (path, ext):
    file_list = []
    for x in os.listdir(path):
        if x.endswith(ext):
            file_list.append(x)

    # sort list to collate pages
    file_list.sort()
    
    return file_list

In [ ]:
# TotalRestart Letter function


In [ ]:
# TotalRestart Calculation function


In [ ]:
# connect to DB
cnx = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor = cnx.cursor()

In [ ]:
# sql query for GAP claims that are RTBP
sql_file = '''
            SELECT c.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1,
                cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                IF(sq.gap_amt_paid > 0, 2,1) AS pymt_type_id, 
                IF(sq.gap_amt_paid > 0, ROUND(cc.gap_payable - sq.gap_amt_paid,2), cc.gap_payable) AS gap_due,
                COALESCE(NULLIF(cb.acct_number,''),'0') AS acct_nbr, c.loss_date
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN claim_calculations cc
                USING (claim_id)
            INNER JOIN claim_status cs
                ON (c.status_id = cs.status_id)
            LEFT JOIN (SELECT cp.claim_id, SUM(cp.payment_amount) AS gap_amt_paid
                    FROM claim_payments cp
                    INNER JOIN (SELECT c.claim_id
                                FROM claims c
                                INNER JOIN claim_status cs
                                    ON (c.status_id = cs.status_id)
                                WHERE cs.status_desc_id = 8) rtbp_sq
                        USING (claim_id)
                    WHERE payment_category_id = 1
                    GROUP BY cp.claim_id) sq
                ON (c.claim_id = sq.claim_id)
            WHERE cs.status_desc_id = 8;
            '''

In [ ]:
# execute sql
cursor.execute(sql_file)

In [ ]:
# save query results as DF
df = pd.DataFrame(cursor.fetchall())

In [ ]:
# add column names
df_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number', 'loss_date']

df.columns = df_cols

In [ ]:
# sql query for PLUS claims that are RTBP
sql_file_plus = '''
                SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, 
                    cl.address1, cl.city, cl.state, cl.zip, cl.payment_method, cb.first, cb.last, 
                    1 AS pymt_type_id, 
                    IF(cl.customer_securityId = 9401, 1500, 1000) AS gap_plus_due, COALESCE(NULLIF(cb.acct_number,''),'0') AS acct_nbr,
                    c.loss_date
                FROM claims c
                INNER JOIN claim_lender cl
                    USING (claim_id)
                INNER JOIN claim_borrower cb
                    USING (claim_id)
                INNER JOIN (SELECT pb.claim_id
                            FROM claim_plus_benefit pb
                            WHERE status_desc_id = 8) sqp
                    USING (claim_id)
                INNER JOIN (SELECT c.claim_id
                            FROM claims c
                            INNER JOIN claim_status cs
                                ON (c.status_id = cs.status_id)  
                            WHERE cs.status_desc_id = 8
                                OR cs.status_desc_id = 4) sqg
                    USING (claim_id);
            '''

In [ ]:
# execute sql
cursor.execute(sql_file_plus)

In [ ]:
# save query results as DF
df2 = pd.DataFrame(cursor.fetchall())

In [ ]:
# add column names
df2_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number', 'loss_date']

df2.columns = df2_cols

In [ ]:
# sql query for TotalRestart claims that are RTBP
sql_file_tr = '''
            SELECT sqp.claim_id, c.claim_nbr, c.carrier_id, cl.alt_name, cl.dealer_securityId, cl.contact, cl.address1, 
            cl.city, cl.state, cl.zip, 'Check' AS payment_method, cb.first, cb.last, 1 AS pymt_type_id, 
            ctr.totalrestart_payable AS tr_due, COALESCE(NULLIF(cb.acct_number,''),'0') AS acct_nbr, c.loss_date
            FROM claims c
            INNER JOIN claim_lender cl
                USING (claim_id)
            INNER JOIN claim_borrower cb
                USING (claim_id)
            INNER JOIN (SELECT pb.claim_id
                        FROM claim_totalrestart pb
                        WHERE status_desc_id = 8) sqp
                USING (claim_id)
            INNER JOIN claim_totalrestart ctr
                USING (claim_id)
             '''

In [ ]:
# execute sql
cursor.execute(sql_file_tr)

In [ ]:
# save query results as DF
df3 = pd.DataFrame(cursor.fetchall())

In [ ]:
# add column names
df3_cols = ['claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                            'pymt_method', 'first', 'last', 'pymt_type_id', 'amount', 'acct_number', 'loss_date']

df3.columns = df3_cols

In [ ]:
# close mysql connection
cursor.close()
cnx.close()

In [ ]:
# get expense account name
cnx = mc.connect(user=mysql_u, password=mysql_pw,
                host=mysql_host,
                database='claim_qb_payments')
cursor = cnx.cursor()

In [ ]:
# GAP
# convert columns list to string
cols = ", ".join(df_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df.iterrows():

    sql_file2 = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", "{loss_date}", 1, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], loss_date = rows['loss_date'], batchId = batch_id)
     
    # execute and commit sql
    cursor.execute(sql_file2)
    cnx.commit()

In [ ]:
# PLUS
# convert columns list to string
cols = ", ".join(df2_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df2.iterrows():
    sql_file2_plus = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", "{loss_date}", 2, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], loss_date = rows['loss_date'], batchId = batch_id)
          
    # execute and commit sql
    cursor.execute(sql_file2_plus)
    cnx.commit()

In [ ]:
# TOTALRESTART
# convert columns list to string
cols = ", ".join(df3_cols)

# insert DF into the ready_to_be_paid table of the claim_qb_payments database
for x,rows in df3.iterrows():
    sql_file2_tr = '''INSERT INTO ready_to_be_paid ({columns}, payment_category_id, check_nbr, batch_id, qb_txnid, toVGC) VALUES ({claim_id}, "{claim_nbr}", {carrier_id},"{lender_name}", "{lender_id}","{contact}", 
                "{address1}", "{city}", "{state}", "{zip}", "{pymt_method}", "{first}", "{last}", {pymt_type_id}, {amount}, "{acct_nbr}", "{loss_date}", 3, 0, {batchId}, 0, 0);'''.format(columns=cols, 
                claim_id=rows['claim_id'], claim_nbr=rows['claim_nbr'], carrier_id=rows['carrier_id'], lender_name=rows['lender_name'], lender_id=rows['dealer_securityId'], 
                contact=rows['contact'], address1=rows['address1'], city=rows['city'], state=rows['state'], zip=rows['zip'], 
                pymt_method=rows['pymt_method'], first = rows['first'], last = rows['last'], pymt_type_id = rows['pymt_type_id'], 
                amount = rows['amount'], acct_nbr = rows['acct_number'], loss_date = rows['loss_date'], batchId = batch_id)
          
    # execute and commit sql
    cursor.execute(sql_file2_tr)
    cnx.commit()

In [ ]:
# create select query to pull current batch with ID
sql_file3 = '''SELECT rtbp_id, claim_id, claim_nbr, carrier_id, lender_name, dealer_securityId, contact, address1, city, 
                     state, zip, pymt_method, first, last, pymt_type_id, amount, payment_category_id, check_nbr, batch_id, 
                     qb_txnid, acct_number, loss_date
              FROM ready_to_be_paid
              WHERE batch_id = {batchId}
              
              ORDER BY payment_category_id, claim_id;'''.format(batchId = batch_id)

In [ ]:
# execute sql
cursor.execute(sql_file3)

In [ ]:
# save query results as DF
pymts_df = pd.DataFrame(cursor.fetchall())

# add column names
pymts_df_cols = ['rtbp_id', 'claim_id', 'claim_nbr', 'carrier_id', 'lender_name', 'dealer_securityId', 'contact', 'address1', 'city', 'state', 'zip', 
                    'pymt_method', 'first', 'last', 'pymt_type_id', 'amount','payment_category_id', 'check_nbr', 'batch_id', 'qb_txnid', 
                    'acct_number', 'loss_date']

pymts_df.columns = pymts_df_cols

In [ ]:
# sql query for expense accounts in DB
sql_file4 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Expense';
    '''

In [ ]:
# execute sql
cursor.execute(sql_file4)
# save query results as DF
expense_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'expense']
expense_df.columns = col_names

In [ ]:
# sql query for checking accounts in DB
sql_file5 = '''
    SELECT carrier_id, qb_fullname
    FROM qb_accounts
    WHERE account_type = 'Checking';
    '''

In [ ]:
# execute sql
cursor.execute(sql_file5)
# save query results as DF
checking_df = pd.DataFrame(cursor.fetchall())
# add column names to DF
col_names = ['carrier_id', 'checking']
checking_df.columns = col_names

In [ ]:
# Merge expense account name into df
pymts_df = pymts_df.merge(expense_df, left_on='carrier_id', right_on='carrier_id').copy()

In [ ]:
# Merge checking account name into df
pymts_df = pymts_df.merge(checking_df, left_on='carrier_id', right_on='carrier_id').copy()

In [ ]:
# Create sql server connection
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+svr+';DATABASE='+db+';UID='+sql_u+';PWD='+ sql_pw)

In [ ]:
# create query
sql_svr_file = '''SELECT VGSecurityId, QB_ListID
                  FROM business_entity
                  WHERE QB_ListID IS NOT NULL;
               '''
# execute query
listid_df = pd.read_sql(sql_svr_file, cnxn) 

In [ ]:
# TEMPORARY ########################################################################################
# Convert test dealer_securityId to Production dealer_securityId
pymts_df['dealer_securityId'].replace({22260:46724,21945:52715}, inplace=True)
# TEMPORARY ########################################################################################

In [ ]:
# TOTALRESTART #####################################################################################
# Update TR claims with the correct 'checking' & 'expense' names and possibly QB_ListID
# maybe change carrier to pull correct 'checking' & 'expense' names and possibly QB_ListID??
####################################################################################################

In [ ]:
# Merge QB_ListID into df
pymts_df = pymts_df.merge(listid_df, left_on='dealer_securityId', right_on='VGSecurityId').copy()

In [ ]:
# Add carrier name
cnx_c = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor_c = cnx_c.cursor()

sql_file6 = '''
    SELECT carrier_id, description
    FROM carriers;
    '''
cursor_c.execute(sql_file6)

carrier_df = pd.DataFrame(cursor_c.fetchall())

col_names = ['carrier_id', 'carrier']
carrier_df.columns = col_names

cursor_c.close()
cnx_c.close()

# Merge QB_ListID into df
pymts_df = pymts_df.merge(carrier_df, left_on='carrier_id', right_on='carrier_id').copy()

In [ ]:
# payments greater than 0 df
qb_pymts_df = pymts_df.loc[pymts_df['amount'] > 0]
qb_pymts_df

In [ ]:
# Connect to Quickbooks
sessionManager = wc.Dispatch("QBXMLRP2.RequestProcessor")    
sessionManager.OpenConnection('', 'Test qbXML Request')
ticket = sessionManager.BeginSession("", 2)

In [ ]:
# create qbxml query to add payments to QB
pay_date = f"{dt.date.today():%Y-%m-%d}"

for index, row in pymts_df.iterrows():
    if row['payment_category_id'] == 1:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Claim Pymt'
        else: 
            pymt_type = 'GAP Claim'

    elif row['payment_category_id'] == 2:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional GAP Plus Pymt'
        else: 
            pymt_type = 'GAP Plus'
            
    elif row['payment_category_id'] == 3:
        if row['pymt_type_id'] == 2:
            pymt_type = 'Additional TotalRestart Pymt'
        else: 
            pymt_type = 'TotalRestart'

    pymtAmt = "{:.2f}".format(row['amount'])

    if row['pymt_method'] == 'Check':
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>true</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)

    elif 'ACH' in row['pymt_method']:
        qbxmlQuery = '''
        <?qbxml version="14.0"?>
        <QBXML>
            <QBXMLMsgsRq onError="stopOnError">
                <CheckAddRq>
                    <CheckAdd>
                        <AccountRef>
                            <FullName>{checking}</FullName>
                        </AccountRef>
                        <PayeeEntityRef>
                            <ListID>{lender_qbid}</ListID>
                        </PayeeEntityRef>
                        <RefNumber>ACH</RefNumber>
                        <TxnDate>{date}</TxnDate>
                        <Memo>{memo}</Memo>
                        <Address>
                            <Addr1>{lender}</Addr1>
                            <Addr2>{contact}</Addr2>
                            <Addr3>{address}</Addr3>
                            <City>{city}</City>
                            <State>{state}</State>
                            <PostalCode>{zip}</PostalCode>
                        </Address>
                        <IsToBePrinted>false</IsToBePrinted>
                        <ExpenseLineAdd>
                            <AccountRef>
                                <FullName>{expense}</FullName>
                            </AccountRef>
                            <Amount>{amount}</Amount>
                            <Memo>{memo}</Memo>
                        </ExpenseLineAdd>
                    </CheckAdd>
                </CheckAddRq>            
            </QBXMLMsgsRq>
        </QBXML>'''.format(checking=row['checking'], lender_qbid=row['QB_ListID'], lender=row['lender_name'], date=pay_date, 
                memo=f"{row['last']}/{row['first']} {pymt_type}", contact=row['contact'], address=row['address1'], city=row['city'], state=row['state'],
                zip=row['zip'], expense=row['expense'], amount = pymtAmt)
        
    # Send query and receive response
    responseString = sessionManager.ProcessRequest(ticket, qbxmlQuery)

    # output TxnID
    QBXML = ET.fromstring(responseString)
    QBXMLMsgsRs = QBXML.find('QBXMLMsgsRs')
    checkResults = QBXMLMsgsRs.iter("CheckRet")
    txnId = 0
    for checkResult in checkResults:
        txnId = checkResult.find('TxnID').text

    # Add TxnID to ready_to_be_paid table
    sql_file6 = '''UPDATE ready_to_be_paid
                   SET qb_txnid = '{TxnID}'
                   WHERE rtbp_id = {rowID};'''.format(TxnID=txnId, rowID=row['rtbp_id'])
    
    # execute and commit sql
    cursor.execute(sql_file6)
    cnx.commit()


In [ ]:
# Disconnect from Quickbooks
sessionManager.EndSession(ticket)
sessionManager.CloseConnection()

In [ ]:
# close mysql connection
cursor.close()
cnx.close()

In [ ]:
# Create and print claim letter and calculation
# Import functions to create letters [payment_category_id = GAP (1), PLUS(2), TOTALRESTART (3)] 

In [ ]:
# Add Fraud Language fields to pymts_df
cnx_f = mc.connect(user=vgc_u, password=vgc_pw,
                host=vgc_host,
                database='visualgap_claims')
cursor_f = cnx_f.cursor()

sql_file7 = '''
    SELECT StateId, 
    StateDesc,
    StateCode,
    CAST(Language AS CHAR(1000) CHARACTER SET utf8)
    FROM FraudLang;
    '''
cursor_f.execute(sql_file7)

fraud_lang_df = pd.DataFrame(cursor_f.fetchall())

col_names = ['StateId', 'StateDesc', 'StateCode', 'f_lang']
fraud_lang_df.columns = col_names

cursor_f.close()
cnx_f.close()

# Merge QB_ListID into df
pymts_df = pymts_df.merge(fraud_lang_df, left_on='state', right_on='StateId').copy()

In [ ]:
pymts_df

In [ ]:
# Update Column names
pymts_df.rename(columns = {'amount':'payment_amount', 'lender_name':'alt_name'}, inplace = True)

Create GAP Letters

In [ ]:
# Collect GAP payments
gap_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 1].copy()


In [ ]:
gap_letters_df = gap_pymts_df.loc[gap_pymts_df['payment_amount'] > 0].copy()

In [ ]:
# Remove files from staging directory
file_staging_dir = './letters/staging/'
file_ext = ".pdf"

clear_dir(file_staging_dir, file_ext)

In [ ]:
# create letters for amounts greater than $0 paid via check

# Filter gap_letters_df for pymt_method 'Check'
checks_df = gap_letters_df.loc[gap_letters_df['pymt_method'] == 'Check'].copy()

if len(checks_df.index) > 0:
    # Create list of Carriers on checks_df
    carriers = []
    carriers = checks_df.carrier.unique()
    letter_cols = ['claim_nbr', 'loss_date', 'alt_name', 'contact', 'address1', 'city', 'state', 'zip', 'first', 'last', 'acct_number', 'payment_amount', 'StateDesc', 'StateCode', 'f_lang' ]

    # Loop through carriers list
    for carrier in carriers:

        # Variable Defaults
        sql_where_cal = ''
        g_letters = True
        s_letters = True
        cals = []

        # check for payment_type_id for GAP letters by carrier
        g_letters_df = checks_df.loc[(checks_df['pymt_type_id'] == 1) & (checks_df['carrier'] == carrier)].copy()

        # # GAP Letter - create WHERE statement
        if len(g_letters_df.index) > 0:
            g_letters = True
        else:
            # No Letters
            g_letters = False

        # check for payment_type_id for Supplemental letters
        s_letters_df = checks_df.loc[(checks_df['pymt_type_id'] == 2) & (checks_df['carrier'] == carrier)].copy()

        # Supplemental Letter
        if len(s_letters_df.index) > 0:
            s_letters = True
        else:
            # No Letters
            s_letters = False

        # Calculations
        # check for calculations for carrier
        calculations_df = checks_df.loc[checks_df['carrier'] == carrier].copy()

        if len(calculations_df.index) > 0:
            # Multiple Calculation
            for index, row in calculations_df.iterrows():
                cals.append(row['claim_id'])

        # Connect to SQL DB
        cnx = mc.connect(user=vgc_u, password=vgc_pw,
                        host=vgc_host,
                        database='visualgap_claims')
        cursor = cnx.cursor()

        # GAP - Create letters
        if g_letters == True:
            # create df for template
            g_template_df = g_letters_df[letter_cols]

            # Create GAP Letters
            g_pdf_template = "letters/pdf_templates/GAP_letter_template.pdf"
            position = 1
            gap_letter(g_template_df, g_pdf_template, position)
        
        # Supplemental - Create letters
        if s_letters == True:
            # create df for template
            s_template_df = s_letters_df[letter_cols]
            
            # Create Supplement Letters
            s_pdf_template = "letters/pdf_templates/Supp_letter_template.pdf"
            position = 2
            gap_letter(s_template_df, s_pdf_template, position)

        # Calculations - Create SQL query, run query, create calculation sheets
        if g_letters == True or s_letters == True:

            # create calculation dataframe
            temp_cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]                             
            c_consolidate_df = pd.DataFrame(columns = temp_cols)

            for cal in cals:
                # create sql for calculation
                c_sql_query = '''
                        SELECT c.claim_nbr, c.loss_date, l.alt_name, l.contact, b.first, b.last, cc.gap_payable,
                            cl.incp_date, cl.last_payment, cl.interest_rate, cl.amount AS Amt_Fin, cc.balance_last_pay,
                            cc.nbr_of_days, cc.per_day, ROUND(cc.payoff,2) AS payoff, (cc.ltv * 100) AS ltv, cc.covered_fin_amount,
                            (cc.percent_uncovered * 100) AS percent_uncovered, (cc.ltv_limit * 100) AS ltv_limit, v.nada_value, CONCAT(v.year, ' ', v.make, ' ', v.model) AS vehicle,
                            v.deductible, cls.description AS loss_type, cc26.description AS primary_carrier,
                            cc14.amount AS past_due, cc15.amount AS late_fees, cc16.amount AS skip_pymts,
                            cc17.amount AS skip_fees, cc5.amount AS primary_pymt, cc7.amount AS excess_deductible,
                            cc8.amount AS scr, cc9.amount AS clr, cc10.amount AS cdr, cc11.amount AS oref,
                            cc18.amount AS salvage, cc19.amount AS prior_dmg, cc20.amount AS over_ltv,
                            cc21.amount AS other1_amt, cc21.description AS other1_description,
                            cc22.amount AS other2_amt, cc22.description AS other2_description, ca.description AS carrier,
                            (cc22.amount + cc21.amount + cc20.amount + cc19.amount + cc18.amount + cc11.amount + cc10.amount + cc9.amount + 
                            cc8.amount + cc7.amount + cc5.amount + cc17.amount + cc16.amount + cc15.amount + cc14.amount) as subtotal
                        FROM claims c
                        INNER JOIN claim_lender l
                            USING (claim_id)
                        INNER JOIN claim_borrower b
                            USING (claim_id)
                        INNER JOIN claim_loan cl
                            USING (claim_id)
                        INNER JOIN claim_calculations cc
                            USING (claim_id)
                        INNER JOIN claim_vehicle v
                            USING (claim_id)
                        INNER JOIN claims_loss_type cls
                            USING (loss_type_id)
                        INNER JOIN carriers ca
                            ON c.carrier_id = ca.carrier_id                        
                        INNER JOIN claim_checklist AS cc26
                            ON c.claim_id = cc26.claim_id
                            AND cc26.checklist_item_id = 26    
                        INNER JOIN claim_checklist AS cc14
                            ON c.claim_id = cc14.claim_id
                            AND cc14.checklist_item_id = 14    
                        INNER JOIN claim_checklist AS cc15
                            ON c.claim_id = cc15.claim_id
                            AND cc15.checklist_item_id = 15    
                        INNER JOIN claim_checklist AS cc16
                            ON c.claim_id = cc16.claim_id
                            AND cc16.checklist_item_id = 16       
                        INNER JOIN claim_checklist AS cc17
                            ON c.claim_id = cc17.claim_id
                            AND cc17.checklist_item_id = 17       
                        INNER JOIN claim_checklist AS cc5
                            ON c.claim_id = cc5.claim_id
                            AND cc5.checklist_item_id = 5
                        INNER JOIN claim_checklist AS cc7
                            ON c.claim_id = cc7.claim_id
                            AND cc7.checklist_item_id = 7
                        INNER JOIN claim_checklist AS cc8
                            ON c.claim_id = cc8.claim_id
                            AND cc8.checklist_item_id = 8
                        INNER JOIN claim_checklist AS cc9
                            ON c.claim_id = cc9.claim_id
                            AND cc9.checklist_item_id = 9
                        INNER JOIN claim_checklist AS cc10
                            ON c.claim_id = cc10.claim_id
                            AND cc10.checklist_item_id = 10
                        INNER JOIN claim_checklist AS cc11
                            ON c.claim_id = cc11.claim_id
                            AND cc11.checklist_item_id = 11      
                        INNER JOIN claim_checklist AS cc18
                            ON c.claim_id = cc18.claim_id
                            AND cc18.checklist_item_id = 18
                        INNER JOIN claim_checklist AS cc19
                            ON c.claim_id = cc19.claim_id
                            AND cc19.checklist_item_id = 19
                        INNER JOIN claim_checklist AS cc20
                            ON c.claim_id = cc20.claim_id
                            AND cc20.checklist_item_id = 20     
                        INNER JOIN claim_checklist AS cc21
                            ON c.claim_id = cc21.claim_id
                            AND cc21.checklist_item_id = 21
                        INNER JOIN claim_checklist AS cc22
                            ON c.claim_id = cc22.claim_id
                            AND cc22.checklist_item_id = 22  
                        WHERE c.claim_id = {claimID};'''.format(claimID = cal)

                cursor.execute(c_sql_query)
                # save query results as DF
                c_temp_df = pd.DataFrame(cursor.fetchall())

                # append query result to c_template_df
                c_consolidate_df = c_consolidate_df.append(c_temp_df)

            cal_cols = {0:'claim_nbr', 1:'loss_date', 2:'alt_name', 3:'contact', 4:'first', 5:'last', 6:'gap_payable', 7:'incp_date', 8:'last_payment', 9:'interest_rate', 10:'Amt_Fin',
                11:'balance_last_pay', 12:'nbr_of_days', 13:'per_day', 14:'payoff', 15:'ltv', 16:'covered_fin_amount', 17:'percent_uncovered', 18:'ltv_limit', 19:'nada_value',
                20:'vehicle', 21:'deductible', 22:'loss_type', 23:'primary_carrier', 24:'past_due', 25:'late_fees', 26:'skip_pymts', 27:'skip_fees', 28:'primary_pymt',
                29:'excess_deductible', 30:'scr', 31:'clr', 32:'cdr', 33:'oref', 34:'salvage', 35:'prior_dmg', 36:'over_ltv', 37:'other1_amt', 38:'other1_description',
                39:'other2_amt', 40:'other2_description', 41:'carrier', 42:'subtotal'}

            c_template_df = c_consolidate_df.rename(columns = cal_cols)
            c_template_df = c_template_df.reset_index(drop=True)

            # Create Calculations
            c_pdf_template = "letters/pdf_templates/GAP_calculation_template.pdf"
            position = 3
            calculations(c_template_df, c_pdf_template, position)
                
        # Close SQL Connection
        cursor.close()
        cnx.close()

        # create 
        file_list = fileList(file_staging_dir, file_ext)

        # Concatenated output file
        outfn = f'S:/claims/letters/{carrier}_{now.strftime("%Y-%m-%d")}_GAP.pdf'

        ConCat_pdf(file_list, outfn)

        # Remove files from staging directory
        clear_dir(file_staging_dir, file_ext)

else: print('No amount greater then 0.')


Create Plus Letters

In [ ]:
# Collect GAP Plus
plus_pymts_df = pymts_df.loc[pymts_df['payment_category_id'] == 2].copy()
# plus_pymts_df.rename(columns = {'amount':'payment_amount', 'lender_name':'alt_name'}, inplace = True)
plus_letters_df = plus_pymts_df.loc[plus_pymts_df['payment_amount'] > 0].copy()

In [ ]:
# GAP plus Letters

# Filter plus_letters_df for pymt_method 'Check'
plus_df = plus_letters_df.loc[plus_letters_df['pymt_method'] == 'Check'].copy()

if len(plus_df.index) > 0:

    # Create list of Carriers on checks_df
    p_carriers = []
    p_carriers = plus_df.carrier.unique()

    # Loop through carriers list
    for p_carrier in p_carriers:

        # Variable Defaults
        p_letters = True

        # check for payment_type_id for GAP letters by carrier
        p_letters_df = plus_df.loc[plus_df['carrier'] == p_carrier].copy()

        # check for records
        if len(p_letters_df.index) > 0:
            p_letters = True
        else:
            # No Letters
            p_letters = False

        # PLUS - Create letters
        if p_letters == True:
            # create df for template
            p_template_df = p_letters_df[letter_cols]

            # Create GAP Letters
            p_pdf_template = "letters/pdf_templates/PLUS_letter_template.pdf"
            position = 1
            gap_letter(p_template_df, p_pdf_template, position)
        
            file_list = fileList(file_staging_dir, file_ext)

            # Concatenated output file
            outfn = f'S:/claims/letters/{p_carrier}_{now.strftime("%Y-%m-%d")}_PLUS.pdf'

            ConCat_pdf(file_list, outfn)

        # Remove files from staging directory
        clear_dir(file_staging_dir, file_ext)

else: print('No Plus.')